In [1]:
#ライブラリのインポート
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#pandasのコラムが100列まで見られる
pd.set_option('display.max_columns', 100)

In [2]:
#データ読み込み
train_df = pd.read_csv('TrainOfHouse.csv', index_col=0)
test_df = pd.read_csv('TestOfHouse.csv', index_col=0)

In [ ]:
#確認用
#train_df.head()
#test_df.head()

In [ ]:
#目的変数、SalePrice
#大きく影響しそうな変数、LotArea,YearBuilt,YearRemodAdd
#ある程度影響しそうな変数、KitchenQual,RoofMatl
#アメリカならではの変数、HeatingQC,PoolArea

In [3]:
#住宅価格(SalePrice)について調べてみる
sns.distplot(train_df['SalePrice'], kde=False)  #ヒストグラム
print(train_df['SalePrice'].describe()) #概要
print(f"歪度: {round(train_df['SalePrice'].skew(), 4)}")  #正規分布と比較した偏り具合、非対称性、0に近いほど左右対称
print(f"尖度: {round(train_df['SalePrice'].kurt(), 4)}")  #正規分布と比較したとがり具合、正だと尖っていて、負だと散らばっている

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64
歪度: 1.8829
尖度: 6.5363


C:\Users\sibuya yuuki\miniconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [ ]:
#データの前処理

In [4]:
#結合
all_df = pd.concat([train_df.drop(columns='SalePrice'), test_df])

In [5]:
#数字が入っているが、予測に考慮しないものはカテゴリ変数(文字列)に変換する
num2str_list = ['MSSubClass', 'YrSold', 'MoSold']
for column in num2str_list:
    all_df[column] = all_df[column].astype(str)

In [6]:
#欠損値の処理(削除or埋める)
#今回は文字列の欠損をNoneで、数字の欠損は０で埋める
for column in all_df.columns:
    if all_df[column].dtype == 'O':                #dtypeがobject
        all_df[column] = all_df[column].fillna('None')
    else:                                          #dtypeがint,float
        all_df[column] = all_df[column].fillna(0)

In [7]:
#特徴量エンジニアリング
#場合によっては、新しい変数を作ったほうが精度が向上する場合も
#ここが自分のアイデアの入れ所！！！

def add_new_columns(df):
    #建物の総面積＝すべての階の面積＋地下の面積
    df['TotalSF'] = df['1stFlrSF'] + df['2ndFlrSF'] + df['TotalBsmtSF']
    
    #一部屋当たりの平均面積＝建物の総面積/部屋数
    df['AreaPerRoom'] = df['TotalSF'] / df['TotRmsAbvGrd']
    
    #築年数＋最新リフォーム年：大きいほど値段が高くなりそう
    df['YearBuiltPlusRemod'] = df['YearBuilt'] + df['YearRemodAdd']
    
    #お風呂の面積
    #Fullbath:浴槽、シャワー、洗面台、便器
    #Halfbath:洗面台、便器、(シャワー)
    #今回はHalfには0.5をかけて評価する
    df['TotalBathrooms'] = df['FullBath'] + (0.5 * df['HalfBath']) + df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath'])
    
    #屋根付きの玄関の総面積
    #Porch:屋根付きの玄関
    df['TotalPorchSF'] = df['OpenPorchSF'] + df['3SsnPorch'] + df['EnclosedPorch'] + df['ScreenPorch'] + df['WoodDeckSF']
    
    #プールの有無
    df['HasPool'] = df['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
    
    #2階の有無
    df['Has2ndFloor'] = df['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
    
    #ガレージの有無
    df['HasGarage'] = df['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
    
    #地下室の有無
    df['HasBsmt'] = df['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
    
    #暖炉の有無
    df['HasFireplace'] = df['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)
    
#コラムの追加
add_new_columns(all_df)

In [8]:
#文字列をカテゴリ変数化
#文字列のままでは、正規化したり機械学習できないのでカテゴリ変数化する
#One-Hot-Encodingを用いる

#pd.get_dummiesを使うとカテゴリ変数化できる
all_df = pd.get_dummies(all_df)
all_df.head()

#上記の変換では説明変数が89個から350個になる
#カテゴリ変数に大小の意味を持たせたい、OneHotEncodingでの説明変数の過剰増加を防ぎたいときは
#OriginalEncoding(LabelEncoding)を用いる

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,TotalSF,AreaPerRoom,YearBuiltPlusRemod,TotalBathrooms,TotalPorchSF,HasPool,Has2ndFloor,HasGarage,HasBsmt,HasFireplace,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,...,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,PoolQC_None,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,MiscFeature_Gar2,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_1,MoSold_10,MoSold_11,MoSold_12,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_None,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,150.0,856.0,856,854,0,1710,1.0,0.0,2,1,3,1,8,0,2003.0,2.0,548.0,0,61,0,0,0,0,0,2566.0,320.750000,4006,3.5,61,0,1,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,284.0,1262.0,1262,0,0,1262,0.0,1.0,2,0,3,1,6,1,1976.0,2.0,460.0,298,0,0,0,0,0,0,2524.0,420.666667,3952,2.5,298,0,0,1,1,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,434.0,920.0,920,866,0,1786,1.0,0.0,2,1,3,1,6,1,2001.0,2.0,608.0,0,42,0,0,0,0,0,2706.0,451.000000,4003,3.5,42,0,1,1,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,540.0,756.0,961,756,0,1717,1.0,0.0,1,0,3,1,7,1,1998.0,3.0,642.0,0,35,272,0,0,0,0,2473.0,353.285714,3885,2.0,307,0,1,1,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
5,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,490.0,1145.0,1145,1053,0,2198,1.0,0.0,2,1,4,1,9,1,2000.0,3.0,836.0,192,84,0,0,0,0,0,3343.0,371.444444,4000,3.5,276,0,1,1,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [9]:
#外れ値の除去
#test_dfからは除去できないので分割して実施
train_df = pd.merge(all_df.iloc[train_df.index[0]: train_df.index[-1]], train_df['SalePrice'], left_index =True, right_index=True)
test_df = all_df.iloc[train_df.index[-1]:]

In [10]:
#外れ値として以下のものを除去する
train_df = train_df[(train_df['LotArea'] < 20000) & (train_df['SalePrice'] < 400000) & (train_df['YearBuilt'] > 1920)]

In [11]:
#住宅価格を対数変換
#データ整理後のSalePriceを可視化、歪度、尖度
sns.distplot(train_df['SalePrice'])
print(f"歪度： {round(train_df['SalePrice'].skew(), 4)}")
print(f"尖度： {round(train_df['SalePrice'].kurt(), 4)}")

歪度： 0.9478
尖度： 0.749


C:\Users\sibuya yuuki\miniconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [13]:
#対数変換する
train_df['SalePriceLog'] = np.log(train_df['SalePrice'])

#変換後のグラフ
sns.distplot(train_df['SalePriceLog'])
print(f"歪度： {round(train_df['SalePriceLog'].skew(), 4)}")
print(f"尖度： {round(train_df['SalePriceLog'].kurt(), 4)}")
#歪度と尖度は0に近いほど良い

歪度： -0.0967
尖度： 0.3187


C:\Users\sibuya yuuki\miniconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [14]:
#3つのデータフレームを作成

#学習データの説明変数
train_X = train_df.drop(columns = ['SalePrice', 'SalePriceLog'])
#学習データの目的変数
train_y = train_df['SalePriceLog']
#予測データの目的変数
test_X = test_df

In [ ]:
#ここまでが前処理
#ここから学習モデルの作成

In [15]:
#ハイパーパラメータのチューニング
#用いる機械学習アルゴリズムによって変わる
def lasso_tuning(train_x, train_y):
    #変数パラメータのリスト
    param_list = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
    
    for cnt, alpha in enumerate(param_list):
        #lasso回帰モデル
        lasso = Lasso(alpha=alpha)
        
        #pipeline作成
        pipeline = make_pipeline(StandardScaler(), lasso)
        
        #学習データ内でホールドアウト検証のために分割
        #テストデータの割合は0.3、seed値は0に固定
        X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3, random_state=0)
        
        #学習
        pipeline.fit(X_train, y_train)
        
        #RMSE(平均誤差)を計算
        train_rmse = np.sqrt(mean_squared_error(y_train, pipeline.predict(X_train)))
        test_rmse = np.sqrt(mean_squared_error(y_test, pipeline.predict(X_test)))
        
        #ベストパラメータの更新
        if cnt == 0:
            best_score = test_rmse
            best_param = alpha
        elif best_score > test_rmse:
            best_score = test_rmse
            best_param = alpha
    
    #ベストパラメータとその時のRMSEを出力
    print('alpha:' + str(best_param))
    print('test score is :' + str(round(best_score, 4)))
    #返却
    return best_param

#best_alphaにベストパラメータをわたす
best_alpha = lasso_tuning(train_X, train_y)

alpha:0.01
test score is :0.0985


In [16]:
#ベストパラメータを用いてモデル作成
lasso = Lasso(alpha = best_alpha)
pipeline = make_pipeline(StandardScaler(), lasso)
pipeline.fit(train_X, train_y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('lasso', Lasso(alpha=0.01))])

In [ ]:
#モデル作成完了
#住宅価格の予測をする

In [17]:
#結果の予測
pred = pipeline.predict(test_X)

In [18]:
#予測結果をプロット
sns.distplot(pred)
print(f'歪度： {round(pd.Series(pred).skew(), 4)}')
print(f'尖度： {round(pd.Series(pred).kurt(), 4)}')

歪度： 0.3733
尖度： 1.0989


C:\Users\sibuya yuuki\miniconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [19]:
#指数変換して元に戻す
pred_exp = np.exp(pred)
#指数変換後をプロット
sns.distplot(pred_exp)
print(f'歪度： {round(pd.Series(pred_exp).skew(), 4)}')
print(f'尖度： {round(pd.Series(pred_exp).kurt(), 4)}')

歪度： 5.4356
尖度： 84.9469


C:\Users\sibuya yuuki\miniconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [20]:
#正規表現に近づける
#ただし、値を変えるのではなく外れ値を削って近づける
pred_exp_ex_outliars = pred_exp[pred_exp<400000]
#400000ドル以上の物件を削る
sns.distplot(pred_exp_ex_outliars)
print(f'歪度： {round(pd.Series(pred_exp_ex_outliars).skew(), 4)}')
print(f'尖度： {round(pd.Series(pred_exp_ex_outliars).kurt(), 4)}')

歪度： 0.999
尖度： 0.807


C:\Users\sibuya yuuki\miniconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [22]:
#学習データのプロット
sns.distplot(train_df['SalePrice'])
print(f"歪度： {round(pd.Series(train_df['SalePrice']).skew(), 4)}")
print(f"尖度： {round(pd.Series(train_df['SalePrice']).kurt(), 4)}")

歪度： 0.9478
尖度： 0.749


C:\Users\sibuya yuuki\miniconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [ ]:
#提出用ファイルの作成
#sample用のデータフレームを利用する
submission_df = pd.read_csv('sample.submission.csv')  
#submission_df.head()
submission_df['SalePrice'] = pred_exp
#エラーが出ないことを確認

In [ ]:
submission_df.to_csv('submission.csv', index=False)